# Agregaciones en Mongo  


#### Nota Previa: En esta parte interesa que tengamos abierta la shell de Mongo

Ya en Python, empezamos por importar la librería que permite cargar datos desde mongo

In [1]:
import pymongo  # la conexión con mongo
from pprint import pprint

In [2]:
# cambiar aquí por el host y el puerto que correspondan
from pymongo import MongoClient
#client = MongoClient('mongodb://172.20.0.11:28001/')
# En local
client = MongoClient('mongodb://127.0.0.1:27017/')


## Introducción

En las bases de datos es muy importante disponer de consultas que permitan combinar diferentes elementos; por ejemplo dada la colección de ventas en una tienda calcular el total vendido; o los subtotales logrados por cada vendedor.
El *pipeline de agregación* es el mecanismo de Mongo para realizar consultas este tipo de consultas complejas. La sintaxis es


#### db.coleccion.aggregate([etapa1, etapa2, .... ] )

No solo se utiliza para realizar agregaciones, puede sustituir completamente a *find*. El argumento que recibe es un array de etapas. Cada etapa realiza una operación concreta. La primera etapa recibe como entrada la colección inicial y genera una *salida1*. La etapa2 recibirá salida1 y generará *salida2* y así sucesivamente.

Cada etapa tiene el aspecto {$operacion:{...}}:

![Stages1](https://gpd.sip.ucm.es/rafa/docencia/images/aggregationfwk.jpg)

La lista de operaciones se puede consultar en (https://docs.mongodb.com/manual/meta/aggregation-quick-reference/#stages). El siguiente gráfico muestra algunas de las etapas más típicas que vamos a ver aquí.

![Stages2](https://webassets.mongodb.com/_com_assets/cms/image04-8a19545722.png)





## 1.- ``$group``

Vamos a comenzar viendo la etapa que equival al "group" de SQL. En esta etapa indicamos el valor por el que queremos agregar en la clave "_id" del dcumento asociado.

### Ejemplo
Vamos a empezar por insertar algunos valores de prueba. Corresponden a diferentes sesiones de *running*. Para cada sesión se tiene nombre del corredor, el mes en el que se realizó, la distancia recorrida y el tiempo en minutos

In [3]:
db = client.running
nombre="nombre"
mes="mes"
distKm="distKm"
tiempoMin="tiempoMin"
db.sesiones.drop()
db.sesiones.insert_many([
    {nombre:"Bertoldo", mes:"Marzo", distKm:6, tiempoMin:42},
    {nombre:"Herminia", mes:"Marzo", distKm:10, tiempoMin:60},
    {nombre:"Bertoldo", mes:"Marzo", distKm:2, tiempoMin:12},
    {nombre:"Herminia", mes:"Marzo", distKm:10, tiempoMin:61},
    {nombre:"Bertoldo", mes:"Abril", distKm:5, tiempoMin:33},
    {nombre:"Herminia", mes:"Abril", distKm:42, tiempoMin:285},
    {nombre:"Aniceto", mes:"Abril", distKm:5, tiempoMin:33}])
print("insertados: ",db.sesiones.count_documents({}))

insertados:  7


Supongamos que queremos saber el número de sesiones que ha realizado cada persona:

In [50]:
r = db.sesiones.aggregate(                                    # aggregate significa que vamos a usar el pipeline
                      [                                       # lista de operaciones, a realizar en secuencia
                       {"$group":                             # en este caso solo una operación, agrupar
                               { "_id":"$nombre",             # agrupamos por nombre
                                 "num_sesiones": {"$sum":1}   # cuenta el num.elementos en el grupo
                               }
                        }
                       ]
                     )
for doc in r: pprint(doc)

{'_id': 'Herminia', 'num_sesiones': 3}
{'_id': 'Bertoldo', 'num_sesiones': 3}
{'_id': 'Aniceto', 'num_sesiones': 1}


### P1 
Consideramos el código

In [5]:
db.coord.drop()
for i in range(5):
    for j in range(4): 
        db.coord.insert_one({"x":i,"y":j+i});

¿Cuántos elementos devolverá la siguiente consulta de agregación?

In [46]:
r =  db.coord.aggregate([ {"$group":{"_id":'1'}} ])
# para obtener la solución
for doc in r: pprint(doc)
# Devilvera 5 elementos, cada uno correspondiente a la agrupación por cada uno de los 5 valores que hemos dado a i

{'_id': 1}


También se puede agrupar por dos claves, en este caso por nombre y mes

In [7]:
r = db.sesiones.aggregate(
 [
    {"$group":
        { "_id":{"nombre":"$nombre",
                 "mes": "$mes"},
          "num_sesiones": {"$sum":1}
        }
    }
  ]
)
for doc in r: pprint(doc)

{'_id': {'mes': 'Marzo', 'nombre': 'Bertoldo'}, 'num_sesiones': 2}
{'_id': {'mes': 'Abril', 'nombre': 'Aniceto'}, 'num_sesiones': 1}
{'_id': {'mes': 'Marzo', 'nombre': 'Herminia'}, 'num_sesiones': 2}
{'_id': {'mes': 'Abril', 'nombre': 'Bertoldo'}, 'num_sesiones': 1}
{'_id': {'mes': 'Abril', 'nombre': 'Herminia'}, 'num_sesiones': 1}


### P2 
¿Cuantos documentos devolverá la siguiente consulta de agregación?

In [10]:
r = db.coord.aggregate([ {"$group":{"_id":{"lax":'$x', "la":'$y'}}} ])
# devuelve 20 elementos, pues hemos introducido las permitaciones de cinco valores de "x" y cuatro valores de "y"
for doc in r: pprint(doc) 

{'_id': {'la': 1, 'lax': 0}}
{'_id': {'la': 5, 'lax': 4}}
{'_id': {'la': 3, 'lax': 0}}
{'_id': {'la': 4, 'lax': 4}}
{'_id': {'la': 0, 'lax': 0}}
{'_id': {'la': 6, 'lax': 3}}
{'_id': {'la': 7, 'lax': 4}}
{'_id': {'la': 3, 'lax': 2}}
{'_id': {'la': 5, 'lax': 2}}
{'_id': {'la': 6, 'lax': 4}}
{'_id': {'la': 4, 'lax': 3}}
{'_id': {'la': 4, 'lax': 2}}
{'_id': {'la': 2, 'lax': 1}}
{'_id': {'la': 5, 'lax': 3}}
{'_id': {'la': 3, 'lax': 3}}
{'_id': {'la': 1, 'lax': 1}}
{'_id': {'la': 4, 'lax': 1}}
{'_id': {'la': 2, 'lax': 2}}
{'_id': {'la': 3, 'lax': 1}}
{'_id': {'la': 2, 'lax': 0}}


#### Funciones de agregación

Ya hemos visto una función de agregación, $sum, pero hay muchas otras:

- ``$``sum: suma (o incrementa)
- ``$``avg : calcula la media
- ``$``min: mínimo de los valores
- ``$``max: máximo
- ``$``push: Mete en un array un valor determinado
- ``$``addToSet: Mete en un array los valore que digamos, pero solo una vez
- ``$``first: obtiene el primer elemento del grupo, a menudo junto con sort
- ``$``last: obtiene el último elemento, a menudo junto con sort


Vamos a verlas una a una:

#### ``$``sum

Ya la hemos visto como función para "contar" usando ``$sum:1``, pero su propósito original es sumar:

In [11]:
r = db.sesiones.aggregate(
[
    {"$group":
        {
            "_id":{"nombre":"$nombre"},
            "num_km": {"$sum":'$distKm'}
        }
    }
])
for doc in r: pprint(doc)

{'_id': {'nombre': 'Bertoldo'}, 'num_km': 13}
{'_id': {'nombre': 'Aniceto'}, 'num_km': 5}
{'_id': {'nombre': 'Herminia'}, 'num_km': 62}


Tenemos el total de kilómetros que ha corrido cada persona.

#### ``$``avg
Calcula la media. Por ejemplo: kilómetros que corre cada persona de media al mes

In [32]:
r =db.sesiones.aggregate([
    {'$group':
        { '_id':{'nombre':"$nombre",'mes': "$mes"},
          'media': {'$avg':'$distKm'}
        }
    }
])

for doc in r: pprint(doc)

{'_id': {'mes': 'Marzo', 'nombre': 'Bertoldo'}, 'media': 4.0}
{'_id': {'mes': 'Abril', 'nombre': 'Herminia'}, 'media': 42.0}
{'_id': {'mes': 'Abril', 'nombre': 'Bertoldo'}, 'media': 5.0}
{'_id': {'mes': 'Marzo', 'nombre': 'Herminia'}, 'media': 10.0}
{'_id': {'mes': 'Abril', 'nombre': 'Aniceto'}, 'media': 5.0}


**P3** ¿Cómo calcular el número medio de sesiones por persona al mes? (es decir, se cuenta el número de sesiones por persona y mes y a continuación se hace la media de este dato)

In [16]:
r =db.sesiones.aggregate([
    {'$group':
        { '_id':{'nombre':"$nombre",'mes': "$mes"},
          'numSesiones': {'$sum':1}
        },

    }
])

for doc in r: pprint(doc)

{'_id': {'mes': 'Marzo', 'nombre': 'Bertoldo'}, 'numSesiones': 2}
{'_id': {'mes': 'Abril', 'nombre': 'Aniceto'}, 'numSesiones': 1}
{'_id': {'mes': 'Marzo', 'nombre': 'Herminia'}, 'numSesiones': 2}
{'_id': {'mes': 'Abril', 'nombre': 'Bertoldo'}, 'numSesiones': 1}
{'_id': {'mes': 'Abril', 'nombre': 'Herminia'}, 'numSesiones': 1}


#### ``$addToSet`` 

Crea arrays agrupando elementos.

Ejemplo: Supongamos que queremos saber qué distancias ha corrido cada persona.
Agrupamos por el nombre y "coleccionamos" las distancias distintas


In [17]:
r = db.sesiones.aggregate(
                      [
                       {'$group':
                               { '_id':{'nombre':"$nombre"},
                                 'distancias': {'$addToSet':'$distKm'}
                               }
                        }
                       ]
                     )
for doc in r: pprint(doc)

{'_id': {'nombre': 'Bertoldo'}, 'distancias': [5, 2, 6]}
{'_id': {'nombre': 'Aniceto'}, 'distancias': [5]}
{'_id': {'nombre': 'Herminia'}, 'distancias': [10, 42]}


``$push``

Análogo a ``$addToSet`` pero admite repeticiones.

** P4 ** queremos saber en cada mes qué distancias se han hecho en alguna sesión. Si una distancia se ha corrido varias veces en ese mes debe aparecer varias veces:

#### ``$unwind``

Es el inverso de ``$push``; cuando tenemos documentos que contienen un array y queremos agrupar por valores del array, a veces conviene eliminar los
arrays y convertirlos en múltiples documentos. En realidad estamos "normalizando" los arrays (primera forma normal).

Volvemos al ejemplo de personas con aficiones:


In [18]:
db.gustos.drop()
db.gustos.insert_one({'nombre':"Bertoldo",  'aficiones':["siesta","cine"]})
db.gustos.insert_one({'nombre':"Herminia",  'aficiones':["correr","cine"]})
db.gustos.insert_one({'nombre':"Aniceta",   'aficiones':["viajar","cine"]})
db.gustos.insert_one({'nombre':"Godofredo", 'aficiones':["correr","montaña", "cine"]})



Queremos saber el número de personas con el que cuenta cada afición. ¿Cómo hacerlo?

Para ello el primer paso es hacer $unwind:

In [19]:
r = db.gustos.aggregate([   {'$unwind':'$aficiones'}   ] )
for doc in r: pprint(doc)

{'_id': ObjectId('63415a1dc704a416d5719ab4'),
 'aficiones': 'siesta',
 'nombre': 'Bertoldo'}
{'_id': ObjectId('63415a1dc704a416d5719ab4'),
 'aficiones': 'cine',
 'nombre': 'Bertoldo'}
{'_id': ObjectId('63415a1dc704a416d5719ab5'),
 'aficiones': 'correr',
 'nombre': 'Herminia'}
{'_id': ObjectId('63415a1dc704a416d5719ab5'),
 'aficiones': 'cine',
 'nombre': 'Herminia'}
{'_id': ObjectId('63415a1dc704a416d5719ab6'),
 'aficiones': 'viajar',
 'nombre': 'Aniceta'}
{'_id': ObjectId('63415a1dc704a416d5719ab6'),
 'aficiones': 'cine',
 'nombre': 'Aniceta'}
{'_id': ObjectId('63415a1dc704a416d5719ab7'),
 'aficiones': 'correr',
 'nombre': 'Godofredo'}
{'_id': ObjectId('63415a1dc704a416d5719ab7'),
 'aficiones': 'montaña',
 'nombre': 'Godofredo'}
{'_id': ObjectId('63415a1dc704a416d5719ab7'),
 'aficiones': 'cine',
 'nombre': 'Godofredo'}


Ahora es fácil pensar en la siguiente etapa: agrupar por aficiones

In [20]:
r = db.gustos.aggregate([
                     {'$unwind':'$aficiones'},
                     {'$group':
                          {'_id':'$aficiones',
                           'total':{'$sum':1} 
                          } 
                     }
                    ])
for doc in r: pprint(doc)

{'_id': 'siesta', 'total': 1}
{'_id': 'viajar', 'total': 1}
{'_id': 'cine', 'total': 4}
{'_id': 'montaña', 'total': 1}
{'_id': 'correr', 'total': 2}


#### ``$max``, ``$min``

El mayor o el menor valor dentro de cada grupo.

**P5** Para cada persona queremos saber la máxima y la mínima distancia que ha recorrido

In [22]:
r =db.sesiones.aggregate([
    {'$group':
        { '_id':{'nombre':"$nombre"},
          'sesionMax': {'$max':"$distKm"},
          'sesionMin': {'$min':"$distKm"},
        },
    }
]) 
for doc in r: pprint(doc)

{'_id': {'nombre': 'Herminia'}, 'sesionMax': 42, 'sesionMin': 10}
{'_id': {'nombre': 'Bertoldo'}, 'sesionMax': 6, 'sesionMin': 2}
{'_id': {'nombre': 'Aniceto'}, 'sesionMax': 5, 'sesionMin': 5}


#### ``$first``, ``$last``

Devuelven el primero, respectivamente el último, elemento de un grupo.




##  2.-  Etapas ``$bucket`` y ``$bucketAuto``

El propósito de estas etapas es agrupar según intervalos de una clave. La estructura de $bucket:

```[javascript]
{
  $bucket: {
      groupBy: expression,
      boundaries: [ lowerbound1, lowerbound2, ... ],
      default: literal,
      output: {
         output1: { <$accumulator expression> },
         ...
         outputN: { <$accumulator expression> }
      }
   }
}
```

Significado:

- La parte groupBy corresponde al _id the $group, es decir especifica la clave por la que queremos agrupar. Debe ser un valor que admita comparaciones (``$leq``,...)
- boundaries es un array que indica los límites por los que agrupar. Por ejemplo [0,20,40] crea dos intervalos: [0,20), [20,40)
- default: Nombre del grupo en el que se incluirán los valores que no encajen. Opcional.
- output: claves a incluir en la salida, si no se incluye ninguna al menos se incluirá por defecto count:{``$sum``:1} 

Ejemplo:

Queremos saber cuántas sesiones hay de distancias cortas (0 a 5 km), medias, (5 a 10), largas (10 a 40) o muy largas (más de 40). 

In [23]:
r = db.sesiones.aggregate( [
  {
    '$bucket': {
      'groupBy': "$distKm",
      'boundaries': [ 0, 5, 10, 40 ],
      'default': "Gran distancia"
    }
  }
] )

for doc in r: pprint(doc)


{'_id': 0, 'count': 1}
{'_id': 5, 'count': 3}
{'_id': 10, 'count': 2}
{'_id': 'Gran distancia', 'count': 1}


Supongamos ahora que queremos además saber quiénes han recorrido estas distancias: 

In [43]:
r = db.sesiones.aggregate( [
  {
    '$bucket': {
      'groupBy': "$distKm",
      'boundaries': [ 0, 5, 10, 40 ],
      'default': "Gran distancia",
      'output': {
        "count": { '$sum': 1 },
        "quienes" : { '$addToSet': "$nombre" }
      }
    }
  }
] )
for doc in r: pprint(doc)


{'_id': 0, 'count': 1, 'quienes': ['Bertoldo']}
{'_id': 5, 'count': 3, 'quienes': ['Aniceto', 'Bertoldo']}
{'_id': 10, 'count': 2, 'quienes': ['Herminia']}
{'_id': 'Gran distancia', 'count': 1, 'quienes': ['Herminia']}


#### ``$bucketAuto``

tiene el mismo significado, pero en este caso no decimos los intervalos; solo cuántos queremos obtener. Sintaxis:

```[javascript]
{
  $bucketAuto: {
      groupBy: expression,
      buckets: number,
      output: {
         output1: { <$accumulator expression> },
         ...
      }
      granularity: string
  }
}
```
La clave buckets indica el número de intervalos que usará. El sistema intenta repartirlas de forma más o menos homogénea, pero lo mejor es definir la granularidad de forma específica (consultar la documentación).  Ejemplo: 

In [24]:
r = db.sesiones.aggregate( [
   {
     '$bucketAuto': {
       'groupBy': "$tiempoMin",
       'buckets': 4
     }
   }
 ] )

for doc in r: pprint(doc)


{'_id': {'max': 42, 'min': 12}, 'count': 3}
{'_id': {'max': 61, 'min': 42}, 'count': 2}
{'_id': {'max': 285, 'min': 61}, 'count': 2}


## 3.- ``$facet``

``$facet`` es complejo y potente, permite agrupar varios pipeline de agregación.

Por ejemplo, supongamos que queremos agrupar las sesiones de entrenamiento por intervalos de tiempo y aparte por intervalos de kilómetros. Una solución podría usar dos aggregate, cada una con su correspondiente $bucket. Sin embargo, podemos hacerlo todo a la vez:



In [25]:
r = db.sesiones.aggregate([ 
  {'$facet': 
    {
     "distancia": [
      { '$bucket':  {
          'groupBy': "$distKm",
          'boundaries': [ 0, 5, 10, 40 ],
          'default': "Gran distancia",
          'output': {
            "count": { '$sum': 1 },
            "quienes" : { '$addToSet': "$nombre" }
      }
    }
    }],
    "tiempo": [
      { '$bucket':  {
          'groupBy': "$tiempoMin",
          'boundaries': [ 0, 30, 60 ],
          'default': "Más de una hora",
          'output': {
            "count": { '$sum': 1 },
            "quienes" : { '$addToSet': "$nombre" }
      }
    }
    }] 
  
  }
 }
])

for doc in r: pprint(doc)


{'distancia': [{'_id': 0, 'count': 1, 'quienes': ['Bertoldo']},
               {'_id': 5, 'count': 3, 'quienes': ['Bertoldo', 'Aniceto']},
               {'_id': 10, 'count': 2, 'quienes': ['Herminia']},
               {'_id': 'Gran distancia', 'count': 1, 'quienes': ['Herminia']}],
 'tiempo': [{'_id': 0, 'count': 1, 'quienes': ['Bertoldo']},
            {'_id': 30, 'count': 3, 'quienes': ['Bertoldo', 'Aniceto']},
            {'_id': 'Más de una hora', 'count': 3, 'quienes': ['Herminia']}]}


Hay que recalcar que se trata de una sola 'etapa' del pipeline!!

## 4.- Etapa ``$project``


Project está al nivel de ``$group`` o de ``$bucket``, es decir es una de las etapas permitidas en aggregate. Resulta muy útil para cambiar nombres de claves, introducir nuevas claves, etc. Se utiliza a menudo para "preparar" la agregación, preprocesando los datos. En ocasiones también se utiliza para crear nuevas colecciones.

Para esta preparación de datos suele usar con los siguientes operadores:

- booleanos: ``$and``, ``$or``, ``$not``
- strings: ``$concat``, ``$toUpper``, ``$toLower``, ``$substr``, ``$strcasecmp``
- operadores aritméticos: ``$abs``, ``$add``, ``$ceil``, ``$divide``, ``$exp``, ``$floor``, ``$ln``, ``$log``, ``$log10``, ``$mod``, ``$multiply``, ``$pow``, ``$sqrt``, ``$substract``, ``$trunc``
- conjuntos (arrays vistos como): ``$setEquals``, ``$setInsersection``, ``$setUnion``, ``$setDifference``, ``$setIsSubset``, ``$anyElementTrue``, ``$allElementsTrue``
- arrays: ``$arrayElementAt``, ``$concatArrays``, ``$filter``, ``$isArray``, ``$size``, ``$slice``
- fechas: ``$datOfYear``, ``$dayOfMonth``, ``$dayOfWeek``, ``$yrear``, ``$month``, ``$week``, ``$hour``, ``$minute``, ``$second``, ``$millisecond``, ``$dateToString``
- condicionales: ``cond``, ``ifnull``
- otros: ``map``, ``let``


Ejemplo:

Queremos disponer de los datos de distancias recorridas en millas, sabiendo que

una milla = 1,60934 km

In [50]:
r = db.sesiones.aggregate(
                      [
                       {'$project':
                               {
                                'distMillas':{'$multiply':['$distKm',1.60934]}
                               }
                        }
                       ]
                     )
for doc in r: pprint(doc)


{'_id': ObjectId('5c92c5245085c103487c6a8c'), 'distMillas': 9.65604}
{'_id': ObjectId('5c92c5245085c103487c6a8d'), 'distMillas': 16.0934}
{'_id': ObjectId('5c92c5245085c103487c6a8e'), 'distMillas': 3.21868}
{'_id': ObjectId('5c92c5245085c103487c6a8f'), 'distMillas': 16.0934}
{'_id': ObjectId('5c92c5245085c103487c6a90'), 'distMillas': 8.0467}
{'_id': ObjectId('5c92c5245085c103487c6a91'), 'distMillas': 67.59228}
{'_id': ObjectId('5c92c5245085c103487c6a92'), 'distMillas': 8.0467}


**Observación**: Observación $project solo incluye las claves que se indiquen, con excepción del _id que se incluye siempre pero se puede eliminar explícitamente el id con _id:0 . Si se quiere que una clave aparezca tal cual se puede poner clave:1 en la proyección.


**Aviso**: A partir de la observación anterior, si ponemos clave:1 incluirá el valor original, pero ¿y si queremos que tenga el valor 1? En este caso y similares es útil $literal; se pondría clave:{``$literal``:1}

** P6 ** Escribir una consulta para que documentos del estilo
```
{nombre:"Bertoldo", mes:"Marzo", distKm:6, tiempoMin:42}
```
Se transformen en:
```
{name:"BERTOLDO", distKm:6}
``` 

(el orden de las claves no importa)

In [28]:
r = db.sesiones.aggregate(
                      [
                       {'$project':
                               {
                                #'name':{"$ToUpper":'$nombre'},
                                 'distKm':'$distKm'
                               }
                        }
                       ]
                     )
for doc in r: pprint(doc)


{'_id': ObjectId('634151f4c704a416d5719a99'), 'distKm': 6}
{'_id': ObjectId('634151f4c704a416d5719a9a'), 'distKm': 10}
{'_id': ObjectId('634151f4c704a416d5719a9b'), 'distKm': 2}
{'_id': ObjectId('634151f4c704a416d5719a9c'), 'distKm': 10}
{'_id': ObjectId('634151f4c704a416d5719a9d'), 'distKm': 5}
{'_id': ObjectId('634151f4c704a416d5719a9e'), 'distKm': 42}
{'_id': ObjectId('634151f4c704a416d5719a9f'), 'distKm': 5}


## 5.- Etapa ``$match``

Filtra elementos. Se puede usar tanto antes de la agregación (sería el where de SQL) como después (sería el having).

Ejemplo:

Queremos obtener la media en kilómetros mensuales de cada corredor, pero solo para aquellos valores medios sobre 5km,

In [33]:
r = db.sesiones.aggregate( [
   {'$group': 
        { 
            '_id':{'nombre':"$nombre", 'mes': "$mes"}, 
            'media':{'$avg':'$distKm'}
        } 
   },
   {"$match":
        {"media":5.0}
   }
])

for doc in r: pprint(doc)


{'_id': {'mes': 'Abril', 'nombre': 'Bertoldo'}, 'media': 5.0}
{'_id': {'mes': 'Abril', 'nombre': 'Aniceto'}, 'media': 5.0}


## 6.- Etapa ``$sort``

Sort se emplea para ordenar los resultados. Hay dos formas de ordenar:

- En memoria: es el método por defecto. Es el más rápido pero tiene como límite 100 Mb en la colección a ordenar
- Disco: más lento, pero sin límites; se obtiene añadiendo una etapa con forma {allowDiskUse:true}

** Observación ** :  ``$match`` y ``$sort`` pueden usar índices, pero solo si se hacen al principio del pipeline

** P7 ** ¿Por qué solo usan los índices si son las primeras etapas?

Ejemplo: en el ejemplo de media de kilómetros por corredor y mes, ordenar por mes.

In [34]:
r = db.sesiones.aggregate(
[
    {'$group':
        { '_id':{'nombre':"$nombre", 'mes': "$mes"},
          'media': {'$avg':'$distKm'} }
    },
    {'$sort': {'_id.mes':1} }
])
for doc in r: pprint(doc)


{'_id': {'mes': 'Abril', 'nombre': 'Herminia'}, 'media': 42.0}
{'_id': {'mes': 'Abril', 'nombre': 'Bertoldo'}, 'media': 5.0}
{'_id': {'mes': 'Abril', 'nombre': 'Aniceto'}, 'media': 5.0}
{'_id': {'mes': 'Marzo', 'nombre': 'Bertoldo'}, 'media': 4.0}
{'_id': {'mes': 'Marzo', 'nombre': 'Herminia'}, 'media': 10.0}


** P8 ** Considera la siguiente colección
```[javascript]
>  db.fun.find()
> { "_id" : 0, "a" : 0, "b" : 0, "c" : 21 }
> { "_id" : 1, "a" : 0, "b" : 0, "c" : 54 }
> { "_id" : 2, "a" : 0, "b" : 1, "c" : 52 }
> { "_id" : 3, "a" : 0, "b" : 1, "c" : 17 }
> { "_id" : 4, "a" : 1, "b" : 0, "c" : 22 }
> { "_id" : 5, "a" : 1, "b" : 0, "c" : 5 }
> { "_id" : 6, "a" : 1, "b" : 1, "c" : 87 }
> { "_id" : 7, "a" : 1, "b" : 1, "c" : 97 }
```
    
¿Cuál será el resultado de c tras la siguiente consulta?

```[javascript]
db.fun.aggregate([
{'$match':{a:0}},
{'$sort':{c:-1}},
{'$group':{'_id':"$a", 'c':{'$first':"$c"}}}
])
```
Solucion: { "_id" : 3, "a" : 0, "b" : 1, "c" : 17 }

## 7.- Etapas ``$skip`` y  ``$limit``

Análogas a las funciones del mismo nombre para *find*, y se usan normalmente en combinación con *sort*. Útiles para obtener el mayor , el primero, etc


Ejemplo: corredor que tiene mayor media absoluta:

In [62]:
r = db.sesiones.aggregate([
    {'$group':
    { '_id':{'nombre':"$nombre"},
      'media': {'$avg':'$distKm'} }
    },
    {'$sort': {'media':-1} },
    {'$limit':1}
]
)
for doc in r: pprint(doc)


{'_id': {'nombre': 'Herminia'}, 'media': 20.666666666666668}


** Observación: **  Al contrario de lo que sucedía con *find* aquí el orden entre  ``$skip`` y ``$limit`` sí influye

** P9 ** Considera la siguiente colección
```[javascript]
>  db.fun.find()
> { "_id" : 0, "a" : 0, "b" : 0, "c" : 21 }
> { "_id" : 1, "a" : 0, "b" : 0, "c" : 54 }
> { "_id" : 2, "a" : 0, "b" : 1, "c" : 52 }
> { "_id" : 3, "a" : 0, "b" : 1, "c" : 17 }
> { "_id" : 4, "a" : 1, "b" : 0, "c" : 22 }
> { "_id" : 5, "a" : 1, "b" : 0, "c" : 5 }
> { "_id" : 6, "a" : 1, "b" : 1, "c" : 87 }
> { "_id" : 7, "a" : 1, "b" : 1, "c" : 97 }
```
    
¿Qué devolverá la consulta de agregación?

```[javascript]
db.fun.aggregate([
        {'$group':{'_id':{'a':"$a", 'b':"$b"}, 'c':{'$max':"$c"}}},
        {'$group':{'_id':"$_id.a", 'c':{'$min':"$c"}}}
])
```

## 8.- ``$out``

Redirige la salida de una agrupación creando una nueva colección. Es muy fácil de utilizar:

In [35]:
db.sesiones.aggregate(
                      [
                       {'$group':
                               { '_id':{'nombre':"$nombre", 'mes': "$mes"},
                                 'sesiones':{'$sum':1}
                               }
                        } ,
                        {'$group':
                                {'_id':'$_id.nombre',
                                 'media':{'$avg':'$sesiones'}
                                }
                        },
                        {'$out': 'sesiones_persona_mes'}
                       ]
                     )

No muestra nada en la salida, porque se ha redireccionado a la nueva colección sesiones_persona_mes. Podemos comprobarlo:

In [36]:
for doc in db.sesiones_persona_mes.find(): pprint(doc)

{'_id': 'Aniceto', 'media': 1.0}
{'_id': 'Bertoldo', 'media': 1.5}
{'_id': 'Herminia', 'media': 1.5}


** Observación: ** Si la colección de salida ya existe la borra (sin avisar, claro) 

** Observación **

## 9.- ``$lookup``


Es una etapa añadida en Mongo 3.2

Sintaxis:

```[javascript]
{
   $lookup:
     {
       from: <collection to join>,
       localField: <field from the input documents>,
       foreignField: <field from the documents of the "from" collection>,
       as: <output array field>
     }
}
```

Ejemplo: utilizando las colecciones "sesiones" y "gustos" definidas en este capítulo, queremos conocer, para la persona que mayor distancia total haya recorrido:

- Su nombre
- La distancia total recorrida
- Sus aficiones

In [37]:
r = db.sesiones.aggregate(
                      [
                       {'$group':
                               { '_id':{'nombre':"$nombre"},
                                 'total':{'$sum':'$distKm'}
                               }
                        },
 
                        {'$sort':{'total':-1}},
 
                        {'$limit':1},
 
                        {'$lookup':{
                            'from'        :'gustos',
                            'localField'  :'_id.nombre',
                            'foreignField': 'nombre',
                            'as'          : 'susGustos' }
                        },
 
                        {'$unwind':'$susGustos'},
 
                        {'$project': {
                            '_id':0,
                            'nombre':'$_id.nombre',
                            'total':1,
                            'aficiones': '$susGustos.aficiones'  }
                        }
 
                       ]
                     )

for doc in r: pprint(doc)


{'aficiones': ['correr', 'cine'], 'nombre': 'Herminia', 'total': 62}


** Observación: ** Es importante entender este ejemplo bien. Podemos copiarlo a la siguiente casilla e irlo ejecutando etapa a etapa (borrando las siguientes)

In [45]:
# para jugar
r = db.sesiones.aggregate(
                      [
                       {'$group':
                               { '_id':{'nombre':"$nombre"},
                                 'total':{'$sum':'$distKm'}
                               }
                        },
 
                        {'$sort':{'total':-1}},
 
                        {'$limit':1},
 
                        {'$lookup':{
                            'from'        :'gustos',
                            'localField'  :'_id.nombre',
                            'foreignField': 'nombre',
                            'as'          : 'susGustos' }
                        },
 
                        {'$unwind':'$susGustos'},
 
                        {'$project': {
                            '_id':0,
                            'nombre':'$_id.nombre',
                            'total':1,
                            'aficiones': '$susGustos.aficiones'  }
                        }
 
                       ]
                     )

for doc in r: pprint(doc)

{'aficiones': ['correr', 'cine'], 'nombre': 'Herminia', 'total': 62}


** P10 ** Pregunta. ¿Se podría quitar el sort y el limit, y a cambio añadir un group con max? Algo parecido a:

```
db.sesiones.aggregate(
                      [
                       {$group:
                               { _id:{nombre:"$nombre"},
                                 total:{$sum:'$distKm'}
                               }
                        },
 
                       {$group:
                               {
                                 _id:null,
                                 mayor:{'$max':'$total'}
                               }
                        } ])
```                    

** Observación: ** El uso de _id:null en el ejemplo anterior es el truco que permite agrupar toda la colección.

** Observación: ** Se puede obtener el plan de una operación de agrupación añadiendo una etapa final {explain:true}


## Map Reduce

Ya fuera de aggregate, tenemos esta forma típica de Hadoop para hacer cálculos agregados
Es un sistema de procesamiento basado en dos etapas:

- **map**: Entrada: un documento. Salida: para cada documento se genera una o varias parejas (clave,valor)
- **reduce**: Entrada: una clave con todos sus valores. Salida: un valor (asociado de forma implícita a la clave de entrada)

Ejemplo:

Partimos de la siguiente colección:

In [72]:
db.frases.drop()
db.frases.insert_one({'_id':1,'frase':"el que sabe no habla"})
db.frases.insert_one({'_id':2,'frase':"el que habla no sabe"})
db.frases.insert_one({'_id':3,'frase':"no me digas que no"})

Queremos contar el número de repeticiones de cada palabra.

Desde la shell de Mongo tecleamos:

``` {javascript}
var mapFunctionFrase = function(){
   x = this.frase.split(" ");
   for (var i=0; i<x.length; i++)
        emit(x[i], 1);
};
var reduceFunctionFrase = function(palabra,cuantas){
     return Array.sum(cuantas);};
 
db.frases.mapReduce(mapFunctionFrase,
                       reduceFunctionFrase,
                       {out: "palabras"}
                      )
 
```

Y la salida será de la forma:

```
> db.palabras.find()
{ "_id" : "digas", "value" : 1 }
{ "_id" : "el", "value" : 2 }
{ "_id" : "habla", "value" : 2 }
{ "_id" : "me", "value" : 1 }
{ "_id" : "no", "value" : 4 }
{ "_id" : "que", "value" : 3 }
{ "_id" : "sabe", "value" : 2 }
```

** Observación **: Hay que hacerlo desde la shell porque espera funciones JavaScript

** Observación **: Se incluye por compatibilidad con otros sistemas big data, pero en Mono es muy leno y poco recomendable, mejor usar el pipeline de agregación


Ejemplo
En el ejemplo anterior, obtener la longitud de la frase más larga en toda la colección:

```[javascript]
var mapFunctionFraseLarga = function(){
        emit("max", this.frase.length);
};
var reduceFunctionFraseLarga = function(clave,longs){
     max = longs[0];
     for (var i=1; i<longs.length;i++)
          if (longs[i]>max)
             max = longs[i];
 
     return max;};
 
db.frases.mapReduce(mapFunctionFraseLarga,
                       reduceFunctionFraseLarga,
                       {out: {inline:1}}
                      )
```

** P11 **

Volvemos a considerar el ejemplo de las sesiones de entrenamiento, queremos saber cuántos kilómetros a recorrido cada persona al mes usando MapReduce. Probarlo desde la shell.


## Vistas

Las vistas se introdujeron en la versión 3.4 de MongoDB. Se puede pensar en una vista como una colección "virtual" que se crea a partir de una consulta.

Características principales:

1 Las vistas se definen a través de una consulta de agregación.

```
    db.createView(view, source, pipeline, collation)
```

    Donde
-        view: Un string con el nombre de la vista a crear.
-        source: Un string con el nombre de la colección en la que se basa.
-        pipeline: la secuencia de agregación que define la vista
-        collation: adaptaciones locales (ver ayuda) 

2     Las vistas son de solo lectura; es decir no podemos insertar/borrar/actualizar datos en una vista. En particular las vistas solo se pueden usar en las instrucciones:
- db.collection.find()
-        db.collection.findOne()
-        db.collection.aggregate()
-        db.collection.count()
-        db.collection.distinct() 

3    Si actualizamos la colección base la vista automáticamente cambia.

4    Se muestran como una colleción más con "show collections".

5    La consulta se almacena en System.views. 